In [3]:
import pandas as pd
import numpy as np

## Background:
Active Duty Service Committments (ADSCs) are career moves for airmen which incurr additional service committments. Each committment (given by a specific reason code) is associated with a specific end date that the committment expires. Taking the Aviation Bonus is an example of an ADSC, and is associated with the reason code '85'.

## The dataset:
The dataset contains data from May 2014 through July 2014. Each row is the data for one Pilot at one time period. 

The columns are:
- ```dod_id```: the ID associated with each airman. This ID will be the same along all time of the dataset.
- ```as_of_date```: the date the data was updated in the system
- ```adsc1``` through ```adsc6```: columns containing each ADSC taken by the airman in their career.
- ```dt_adsc1``` through ```dt_adsc6```: the ADSC end date associated with each ADSC incurred by each airman. 

Data for ADSCs are shuffled down their ordered columns when the airman incurrs a new ADSC. For example, if someone took ADSC with code '19', this would go into ```adsc1``` until they took another ADSC, which would bump '19' to ```adsc2``` and the more recent code would populate ```adsc1```. The same organization is used for the date columns. See example below.

In [4]:
example_data = {'dod_id':[1,1,1], 'adsc1':['19','19','85'], 'adsc2':[np.nan,np.nan,'19'], 
                'dt_adsc1':[20190102,20190102,20240505], 'dt_adsc2':[np.nan,np.nan,20190102]}
example = pd.DataFrame(data=example_data)
example

,dod_id,adsc1,adsc2,dt_adsc1,dt_adsc2
0,1,19,NaN,20190102,NaN
1,1,19,NaN,20190102,NaN
2,1,85,19,20240505,20190102.0


## Import the dataset:

In [41]:
adsc_df = pd.read_csv('adsc_data2.csv', low_memory=False)

# Drop rows that has NaN values on selected columns
adsc_df=adsc_df.dropna(subset=['dod_id','as_of_date'])
adsc_df.to_csv('adsc_clean.csv',mode='w',index=False)

adsc_df = pd.read_csv('adsc_clean.csv', low_memory=False)

## Clean the dataset:
- change dates to a common format
- make sure data types are consistent

In [44]:
#adsc_df.info()
Dup_Rows = adsc_df[adsc_df.duplicated()]
print("\n\nDuplicate Rows : \n {}".format(Dup_Rows))
DF_RM_DUP = adsc_df.drop_duplicates(subset=['dod_id'],keep = 'last')
#print('\n\nResult DataFrame after duplicate removal :\n', DF_RM_DUP.head(5))
DF_RM_DUP.to_csv('cleandata.csv',mode='w',index=False)
DF_RM_DUP.dtypes



Duplicate Rows : 
 Empty DataFrame
Columns: [dod_id, as_of_date, dt_adsc1, dt_adsc2, dt_adsc3, dt_adsc4, dt_adsc5, dt_adsc6, adsc1, adsc2, adsc3, adsc4, adsc5, adsc6]
Index: []


dod_id          int64
as_of_date     object
dt_adsc1      float64
dt_adsc2      float64
dt_adsc3      float64
dt_adsc4      float64
dt_adsc5      float64
dt_adsc6      float64
adsc1         float64
adsc2         float64
adsc3         float64
adsc4         float64
adsc5         float64
adsc6         float64
dtype: object

## Feature Engineer Aviation Bonus Columns:
Create a new column called ```took_avb``` which indicates whether or not someone has taken the Aviation Bonus (indicated with code '85') so far in their career. This should be a boolean column. After they have taken the bonus, they should always be marked as having taken it, no matter if the 85 code gets pushed to adsc6 or even pushed out of the data. See example below, pretend the rows of data are in chronilogical order from oldest row of data at thew top. 


Hint: this data is not clean, and there may be gaps in the data along a person's career that need to be taken into account.
Hint 2: not everyone will accept the bonus!

In [7]:
example_data = {'dod_id':[1,1,1,1,1,1], 'adsc1':['19','19','85','85','82','65'],'adsc2':['','','19','19','85','82'],
                'adsc3':['','','','','19','85'],'took_avb':[False, False, True, True,True,True]}
example = pd.DataFrame(data=example_data)
example

# Using 'Address' as the column name
# and equating it to the list
#df['Address'] = address

,dod_id,adsc1,adsc2,adsc3,took_avb
0,1,19,,,False
1,1,19,,,False
2,1,85,19,,True
3,1,85,19,,True
4,1,82,85,19,True
5,1,65,82,85,True


In [8]:
adsc_df["took_avb"] = adsc_df.adsc1 > 85

Create another column called ```took_avb_date``` which indicates the date at which someone took the aviation bonus. This should be the date at which a person's data is updated to hold the 85 reason code. As with the ```took_avb``` column, this ```took_avb_date``` column should be null until someone accepts the bonus.


Create a third column called ```avb_committment_end_date``` which indicates the end date of the aviation bonus committment, and like the other columns, should be null until someone accepts the bonus.